In [63]:
import pandas as pd
import datetime
import numpy as np

In [2]:
ana_clust = pd.read_csv('series_ana_clusters.csv', sep=',')
ana_clust.nunique()
ana_clust.sort_values(by='cluster')

,Code,cluster,Latitude,Longitude,2005-01-01 00:00:00,2005-01-02 00:00:00,2005-01-03 00:00:00,2005-01-04 00:00:00,2005-01-05 00:00:00,2005-01-06 00:00:00,...,2021-02-19 00:00:00,2021-02-20 00:00:00,2021-02-21 00:00:00,2021-02-22 00:00:00,2021-02-23 00:00:00,2021-02-24 00:00:00,2021-02-25 00:00:00,2021-02-26 00:00:00,2021-02-27 00:00:00,2021-02-28 00:00:00
271,2045013,0.0,-20.0844,-45.2967,0.0,23.4,0.0,3.2,36.3,66.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,1945008,0.0,-19.7425,-45.2550,0.0,56.1,1.0,0.0,74.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,1944049,0.0,-19.4283,-44.7197,0.0,5.8,0.0,0.0,3.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,1149000,1.0,-11.3389,-49.2653,0.0,0.0,19.8,0.0,0.0,45.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,1149002,1.0,-11.7372,-49.1361,0.0,0.0,5.7,0.0,10.5,19.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,2247054,125.0,-22.3008,-47.1336,0.0,7.2,14.3,8.6,19.5,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,2246003,125.0,-22.0378,-46.9650,0.0,0.4,0.5,1.7,21.0,21.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,2046013,126.0,-20.3300,-46.3661,9.7,31.3,0.3,15.8,39.3,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,1244022,127.0,-12.2325,-44.3872,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Algoritmo
## Devem ser duas funções, uma para fazer a iteração buscando valores nulos e outra para fazer o processo (<filling_in()>) 
- A função deverá procurar por valores nulos (Px). 
    - SE encontrar:
        - Localize data, cluster e code para input na função <filling_in()>
        - Variáveis:     
            - nP (número de postos)
            - LP (Lista de precipitações observadas em cada posto) 
            - LM (Lista de médias anuais em cada posto)
            - Mx (Média anual do posto em questão)
        - Aplicar a fórmula: Px = (Mx/nP) * (LP[0]/LM[0] + LP[1]/LM[1] + ... + LP[nP-1]/LM[nP-1])

In [47]:
                                # exemplo do cluster 0:

cluster0 = ana_clust.where(ana_clust['cluster']==0).dropna(how='all').set_index('Code')
# n° de postos presentes (nP)
nP0 = cluster0.shape[0] 

cluster_T0 = cluster0.drop(['cluster','Latitude','Longitude'],axis=1).transpose()
cluster_TY0 = cluster_T0.set_index(pd.to_datetime(cluster_T0.index.rename('date')))
cluster_Mean0 = cluster_TY0.resample('y').mean()
cluster_Mean0.set_index(cluster_Mean0.index.year, inplace=True)

# média (Mx)
cluster_Mean0[1944049].loc[2005] 

# Lista da precipitação observada nos outros postos (LP) 
LP0 = cluster_TY0.drop(1944049, axis=1).loc['2005-01-02'].values  

# Lista da precipitação média anual nos outros postos (LM)
LM0 = cluster_Mean0.drop(1944049, axis=1).loc[2005].values

# Px = (Mx/nP) * (LP[0]/LM[0] + LP[1]/LM[1] + ... + LP[nP-1]/LM[nP-1])
# Prefiri fazer um loop pra calcular esse fator resultante do somatório

sequence = np.arange(0,nP0-1)
factor = []
for cada in sequence:
    factor.append(LP0[cada]/LM0[cada])
factor = factor.sum()

Px = (Mx/nP) * factor


In [45]:
                            #   Esboço da função que será chamada ao iterar buscando os Nan

#apply(filling_in())... 
def filling_in(df, date, cluster, code):
    cluster0 = ana_clust.where(ana_clust['cluster']==0).dropna(how='all').set_index('Code')
    # n° de postos presentes (nP)
    nP = cluster0.shape[0]

    cluster_T0 = cluster0.drop(['cluster','Latitude','Longitude'],axis=1).transpose()
    cluster_TY0 = cluster_T0.set_index(pd.to_datetime(cluster_T0.index.rename('date')))

    # Lista da precipitação observada nos outros postos (LP) 
    LP = cluster_TY0.drop(code, axis=1).loc[str(date)].values

    cluster_Mean0 = cluster_TY0.resample('y').mean()
    cluster_Mean0.set_index(cluster_Mean0.index.year, inplace=True)

    # média (Mx)
    Mx = cluster_Mean0[code].loc[date.year]

    # Lista da precipitação média anual nos outros postos (LM)
    LM = cluster_Mean0.drop(1944049, axis=1).loc[2005].values

    # Px = (Mx/nP) * (LP[0]/LM[0] + LP[1]/LM[1] + ... + LP[nP-1]/LM[nP-1])
    # Prefiri fazer um loop pra calcular esse fator resultante do somatório

    sequence = np.arange(0,nP-1)
    factor = []
    for cada in sequence:
        factor.append(LP[cada]/LM[cada])
    factor = factor.sum()

    Px = (Mx/nP) * factor
    return Px